In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext, Row
import pyspark.sql.functions as f
from pyspark import SparkContext
from sparknlp.pretrained import PretrainedPipeline
from sparknlp import Finisher
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from sparknlp.base import *
from sparknlp.annotator import *
import pandas as pd
import re
import os

In [2]:
# spark = SparkSession.builder\
#    .appName("PythonNLPAPPLIER")\
#    .getOrCreate()

# sqlContext = SQLContext(spark)
spark = SparkSession.builder \
    .appName("nlp converter")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "300") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.6")\
    .getOrCreate()
sqlContext = SQLContext(spark)
spark

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1108)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1094)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:579)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1647)
	at org.apache.spark.SparkContext.$anonfun$new$13(SparkContext.scala:514)
	at org.apache.spark.SparkContext.$anonfun$new$13$adapted(SparkContext.scala:514)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:514)
	at org.apache.spark.api.java.JavaSparkC

In [3]:
# path = r"files\raw_texts.feather"
# pandas_df =  pd.read_feather(source+path).drop_duplicates(subset = 'URL', keep = 'first').reset_index(drop=True)
# pandas_df = pandas_df[pandas_df['URL_TEXT']!=""]
# spark_df = sqlContext.createDataFrame(pandas_df)

source = os.path.dirname(os.path.realpath('__file__')).split("src")[0]
path = r"files\raw_texts.parquet"
spark_df = spark.read.parquet(source+path).where(f.col("URL_TEXT")!= "")
spark_df

DataFrame[CLASS: string, KEYWORD: string, DOMAIN: string, URL: string, URL_TEXT: string]

In [5]:
xml = ["(?:<from.*?>)(.*?)(?:<\\/from>)"]
html = ["<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"]        
random_pattern = ['^@.*\{.*\}', '^\..*\{.*\}','\s\s+','\xa0','dbx707', '\xe2','\x80',"\x8b", r"\{\{\.*\}\}", "\x9d", "\u200b"]# only digits: r'\b[0-9]+\b\s*'
url = ["^https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[äöüßa-zA-Z0-9()]{1,6}\\b(?:[-a-zäöüßA-Z0-9()@:%_\\+.~#?&\\/=]*)$", "www\w*de","www\w*com"]
email = ["^\S+@\S+\.\S+$"]
zip = ["^[0-9]{5}(?:-[0-9]{4})?\s?\w*$"]
phone = ["^\\+?[1-9][0-9]{7,14}$"]
dates = ["^[0-9]{1,2}\\/[0-9]{1,2}\\/[0-9]{4}$","^[0-9]{1,2}\\-[0-9]{1,2}\\-[0-9]{4}$", "^[0-9]{4}\\-[0-9]{1,2}\\-[0-9]{1,2}$"]
website_stopwords = ["explore","allgemeine geschäftsbedingungen","allgemein\*",'richtlinie\w*',"\w*recht\w* hinweis\w*","\w*recht\w*","\w*datenschutz\w*", "privacy","policy\w*","cooky\w*","cookie\w*","content\w*"," to ",\
        "anmeld\w*",  "abmeld\w*", "login","log in","logout", "log out", "kunden login", "online","zurück","back","start","select\w*", "ausw\w*","close",\
            "extras","news","report\w*","impressum","newsletter\w*", "owner","internet", "website\w*", "email\w*", "e-mail\w*", "mail\w*", "isbn", "issn",\
                "produkte", "partner","übersicht", "veranstaltungen", "suche\w*","kauf\w*", "angebot\w*", "konfigur\w*", "configur\w*","nutzer\w*","icon\w*",\
                    "zubehör", "garantie", "mehr", "modell\w*", "kontakt\w*","contact\w*","anfrage\w*","skip",'useful links','link\w*',"pin\w*","passw\w*", "password\w*",\
                        "buchen","book" "anfahrt", "finanzdienstleistung\w*" "connected", "required", "sitemap\w*", "\w*\s?abo\w*", 'social media', "socialmedia",\
                            "englisch", "english","deutsch","german","google", "wikipedia", "navigation","\w*shop\w*", "\w*magazin\w*", "lifestyle",\
                                "facebook\w*", "youtube\w*","instagram\w*","xing\w*","linkedin\w*", "blog\w*","spiegel\w*","twitter\w*","sms","video"\
                                    "archiv\w*", "artikel\w*", "article\w*","side\w*", "seite\w*","site","app\w*","\s?abgerufen\s?\w*\s*\d*",\
                                        "januar", "februar", "märz", "april", "mai", "juni", "juli", "august", "september", "oktober", "november", "dezember",\
                                            "dbx707", "db11","\w*\s?straße\s?\d*","\w*\s?strasse\w*", "tel\w*", "\w*\s?download\w*",\
                                                "covid\w*\s?\d*", "corona\w*\s?\d*"]
                                
domain_stopwords = ["(g/km)","use case\w*", "unternehme\w*", "gmbh", "cokg", "co kg", "consult\w*", "handel\w*", "händler\w*", "leistung\w*"]
numbers_only = ["^\\d+$","^\s?[0-9]+(\s+[0-9]+)*\s?$", "\(.*\)","\[.*\]", "^\d+.\d+"," \\d+ "]
special_characters = ['[^äöüßA-Za-z0-9 ]+']#['[\(,.:\);^]']
short_words = ['^\.{0,3}$']

all_pattern_to_remove = email+xml+html+random_pattern+url+zip+phone+dates+website_stopwords+domain_stopwords+numbers_only+special_characters+short_words
spark_df = spark_df.withColumnRenamed("URL_TEXT", 'text')
spark_df = spark_df.withColumn('text', f.lower(f.col('text')))
spark_df = spark_df.withColumn('text', f.regexp_replace(f.col('text'), '\r+|\n+|\t+/', ''))
spark_df = spark_df.withColumn('text', f.regexp_replace(f.col('text'),'[\|]+', ' '))
spark_df = spark_df.withColumn('text', f.regexp_replace(f.col('text'),'\s\s+', ' '))
for pattern in all_pattern_to_remove:
    spark_df = spark_df.withColumn("text", f.regexp_replace(f.col('text'), pattern, ""))
spark_df = spark_df.withColumn('text', f.regexp_replace(f.col('text'),'\s\s+', ' ')) 

# regexTokenizer = RegexTokenizer(pattern='\w+', inputCol="text", outputCol="words")
# spark_df = regexTokenizer.transform(spark_df)
spark_df.show(truncate = False)

+-------+------------+---------------------+---------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

language_detector = LanguageDetectorDL.pretrained("ld_wiki_tatoeba_cnn_21")\
    .setInputCols(["document"])\
    .setOutputCol("lang")\
    .setThreshold(0.8)\
    .setCoalesceSentences(True)

sentenceDetector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("Sentence")

regexTokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

finisher = Finisher() \
    .setInputCols(["token"]) \
    .setIncludeMetadata(True)

pipeline = Pipeline() \
.setStages([
    documentAssembler,
    language_detector,
    sentenceDetector,
    regexTokenizer,
    finisher
    ])

result = pipeline.fit(spark_df).transform(spark_df)

In [ ]:
spark_df.head()